In [1]:
import numpy as np

from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU

Using TensorFlow backend.


In [2]:
import pandas as pd
import pandas_datareader.data as web

import datetime

start = datetime.datetime(2015, 1, 1)

TICKER = 'FDX'
DAILY_DATA = web.DataReader(TICKER, "google", start)
VIX = web.DataReader("VIXCLS", "fred", start)

_DATA = pd.concat([DAILY_DATA, VIX], axis=1).dropna()
_DATA['RETURN'] = _DATA.Close.pct_change()
_DATA['sVIXCLS'] = StandardScaler().fit_transform(_DATA.VIXCLS.values.reshape(_DATA.VIXCLS.shape[0],1))
_DATA['sVolume'] = StandardScaler().fit_transform(_DATA.Volume.values.reshape(_DATA.Volume.shape[0],1))
_DATA = _DATA[1:]
_DATA.head()

,Open,High,Low,Close,Volume,VIXCLS,RETURN,sVIXCLS,sVolume
2015-01-05,171.82,171.93,169.35,169.80,1726204.0,19.92,-0.015367,1.058288,0.032619
2015-01-06,172.42,173.32,169.58,169.79,2613704.0,21.12,-0.000059,1.344465,1.082238
2015-01-07,171.97,172.34,170.05,171.04,1835878.0,19.31,0.007362,0.912814,0.162327
2015-01-08,173.18,175.36,172.46,175.04,1819587.0,17.01,0.023386,0.364307,0.143060
2015-01-09,174.51,175.23,172.35,172.66,1396181.0,17.55,-0.013597,0.493087,-0.357689


In [3]:
steps = 30
horizon = 30
PREDICTORS = ['RETURN','sVIXCLS','sVolume']

TRAIN = _DATA.iloc[:-(steps + horizon)]
TEST = _DATA.iloc[-(steps + horizon):]

In [4]:
TEST_X = []
TEST_Y = []

for i in range( TEST.shape[0] -(steps + horizon) + 1 ):
    TEST_X.append( np.array(TEST.iloc[i:i + steps][PREDICTORS].T ) )
    TEST_Y.append( np.array(TEST.iloc[i + steps:i + steps + horizon ][PREDICTORS].T ) ) 
    
TEST_X = np.array(TEST_X)
#TEST_X = TEST_X.reshape(TEST_X.shape[0], 1, steps)

TEST_Y = np.array(TEST_Y)
#TEST_Y = TEST_Y.reshape(TEST_Y.shape[0], 1, horizon)

TEST_X.shape

(1, 3, 30)

In [5]:
TRAIN_X = []
TRAIN_Y = []

for i in range( TRAIN.shape[0] -(steps + horizon) + 1 ):
    TRAIN_X.append( np.array(TRAIN.iloc[i:i + steps][PREDICTORS].T ) )
    TRAIN_Y.append( np.array(TRAIN.iloc[i + steps:i + steps + horizon ][PREDICTORS].T ) )
    
TRAIN_X = np.array(TRAIN_X)
#TRAIN_X = TRAIN_X.reshape(TRAIN_X.shape[0], 1, steps)

TRAIN_Y = np.array(TRAIN_Y)
#TRAIN_Y = TRAIN_Y.reshape(TRAIN_Y.shape[0], 1, horizon )

TRAIN_X.shape

(488, 3, 30)

In [46]:
# Build Model
model = Sequential()  
model.add(GRU(steps, input_shape=(TRAIN_X.shape[1], TRAIN_X.shape[2]), return_sequences=True, activation='relu', name='GRU_1') )
model.add(GRU(steps + 10, return_sequences=True, activation='relu', name='GRU_2'))
model.add(GRU(steps + 10, return_sequences=True, activation='relu', name='GRU_3'))
model.add(GRU(horizon, return_sequences=True, activation='linear', name='GRU_4'))
#model.add(Dense(horizon, activation='linear'))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.models import load_model

model.load_model('{}_MODEL.h5'.format(TICKER), overwrite=True)

## Fit the model

In [128]:
model.fit(TRAIN_X, TRAIN_Y, epochs=5000, validation_data=(TEST_X, TEST_Y), batch_size=TRAIN_X.shape[0])

Train on 488 samples, validate on 1 samples
Epoch 1/5000
488/488 [==============================] - 0s - loss: 0.0325 - acc: 0.4529 - val_loss: 1.8127 - val_acc: 0.0000e+00
Epoch 2/5000
488/488 [==============================] - 0s - loss: 0.0326 - acc: 0.4563 - val_loss: 1.8713 - val_acc: 0.0000e+00
Epoch 3/5000
488/488 [==============================] - 0s - loss: 0.0330 - acc: 0.4501 - val_loss: 1.7970 - val_acc: 0.0000e+00
Epoch 4/5000
488/488 [==============================] - 0s - loss: 0.0328 - acc: 0.4624 - val_loss: 1.8781 - val_acc: 0.0000e+00
Epoch 5/5000
488/488 [==============================] - 0s - loss: 0.0326 - acc: 0.4563 - val_loss: 1.7956 - val_acc: 0.0000e+00
Epoch 6/5000
488/488 [==============================] - 0s - loss: 0.0321 - acc: 0.4604 - val_loss: 1.8624 - val_acc: 0.0000e+00
Epoch 7/5000
488/488 [==============================] - 0s - loss: 0.0317 - acc: 0.4570 - val_loss: 1.8214 - val_acc: 0.0000e+00
Epoch 8/5000
488/488 [==============================]

488/488 [==============================] - 0s - loss: 0.0336 - acc: 0.4563 - val_loss: 1.8272 - val_acc: 0.0000e+00
Epoch 64/5000
488/488 [==============================] - 0s - loss: 0.0332 - acc: 0.4501 - val_loss: 1.8122 - val_acc: 0.0000e+00
Epoch 65/5000
488/488 [==============================] - 0s - loss: 0.0340 - acc: 0.4583 - val_loss: 1.8526 - val_acc: 0.0000e+00
Epoch 66/5000
488/488 [==============================] - 0s - loss: 0.0346 - acc: 0.4467 - val_loss: 1.8119 - val_acc: 0.0000e+00
Epoch 67/5000
488/488 [==============================] - 0s - loss: 0.0349 - acc: 0.4542 - val_loss: 1.8131 - val_acc: 0.0000e+00
Epoch 68/5000
488/488 [==============================] - 0s - loss: 0.0341 - acc: 0.4501 - val_loss: 1.8407 - val_acc: 0.0000e+00
Epoch 69/5000
488/488 [==============================] - 0s - loss: 0.0336 - acc: 0.4590 - val_loss: 1.7942 - val_acc: 0.0000e+00
Epoch 70/5000
488/488 [==============================] - 0s - loss: 0.0330 - acc: 0.4529 - val_loss: 1.8

488/488 [==============================] - 0s - loss: 0.0318 - acc: 0.4549 - val_loss: 1.8310 - val_acc: 0.0000e+00
Epoch 126/5000
488/488 [==============================] - 0s - loss: 0.0319 - acc: 0.4590 - val_loss: 1.8382 - val_acc: 0.0000e+00
Epoch 127/5000
488/488 [==============================] - 0s - loss: 0.0328 - acc: 0.4570 - val_loss: 1.8231 - val_acc: 0.0000e+00
Epoch 128/5000
488/488 [==============================] - 0s - loss: 0.0332 - acc: 0.4570 - val_loss: 1.8532 - val_acc: 0.0000e+00
Epoch 129/5000
488/488 [==============================] - 0s - loss: 0.0348 - acc: 0.4536 - val_loss: 1.8089 - val_acc: 0.0000e+00
Epoch 130/5000
488/488 [==============================] - 0s - loss: 0.0346 - acc: 0.4570 - val_loss: 1.8588 - val_acc: 0.0000e+00
Epoch 131/5000
488/488 [==============================] - 0s - loss: 0.0359 - acc: 0.4454 - val_loss: 1.7973 - val_acc: 0.0000e+00
Epoch 132/5000
488/488 [==============================] - 0s - loss: 0.0349 - acc: 0.4549 - val_lo

488/488 [==============================] - 0s - loss: 0.0310 - acc: 0.4577 - val_loss: 1.8231 - val_acc: 0.0000e+00
Epoch 188/5000
488/488 [==============================] - 0s - loss: 0.0310 - acc: 0.4556 - val_loss: 1.8166 - val_acc: 0.0000e+00
Epoch 189/5000
488/488 [==============================] - 0s - loss: 0.0311 - acc: 0.4604 - val_loss: 1.8159 - val_acc: 0.0000e+00
Epoch 190/5000
488/488 [==============================] - 0s - loss: 0.0311 - acc: 0.4563 - val_loss: 1.8292 - val_acc: 0.0000e+00
Epoch 191/5000
488/488 [==============================] - 0s - loss: 0.0311 - acc: 0.4583 - val_loss: 1.8195 - val_acc: 0.0000e+00
Epoch 192/5000
488/488 [==============================] - 0s - loss: 0.0312 - acc: 0.4563 - val_loss: 1.8285 - val_acc: 0.0000e+00
Epoch 193/5000
488/488 [==============================] - 0s - loss: 0.0312 - acc: 0.4577 - val_loss: 1.8279 - val_acc: 0.0000e+00
Epoch 194/5000
488/488 [==============================] - 0s - loss: 0.0313 - acc: 0.4556 - val_lo

488/488 [==============================] - 0s - loss: 0.0315 - acc: 0.4563 - val_loss: 1.8233 - val_acc: 0.0000e+00
Epoch 250/5000
488/488 [==============================] - 0s - loss: 0.0316 - acc: 0.4577 - val_loss: 1.8255 - val_acc: 0.0000e+00
Epoch 251/5000
488/488 [==============================] - 0s - loss: 0.0315 - acc: 0.4549 - val_loss: 1.8258 - val_acc: 0.0000e+00
Epoch 252/5000
488/488 [==============================] - 0s - loss: 0.0315 - acc: 0.4597 - val_loss: 1.8283 - val_acc: 0.0000e+00
Epoch 253/5000
488/488 [==============================] - 0s - loss: 0.0315 - acc: 0.4583 - val_loss: 1.8284 - val_acc: 0.0000e+00
Epoch 254/5000
488/488 [==============================] - 0s - loss: 0.0317 - acc: 0.4583 - val_loss: 1.8253 - val_acc: 0.0000e+00
Epoch 255/5000
488/488 [==============================] - 0s - loss: 0.0316 - acc: 0.4542 - val_loss: 1.8363 - val_acc: 0.0000e+00
Epoch 256/5000
488/488 [==============================] - 0s - loss: 0.0317 - acc: 0.4597 - val_lo

...

488/488 [==============================] - 0s - loss: 0.0305 - acc: 0.4624 - val_loss: 1.8450 - val_acc: 0.0000e+00
Epoch 4714/5000
488/488 [==============================] - 0s - loss: 0.0306 - acc: 0.4624 - val_loss: 1.8725 - val_acc: 0.0000e+00
Epoch 4715/5000
488/488 [==============================] - 0s - loss: 0.0308 - acc: 0.4590 - val_loss: 1.8483 - val_acc: 0.0000e+00
Epoch 4716/5000
488/488 [==============================] - 0s - loss: 0.0310 - acc: 0.4611 - val_loss: 1.8819 - val_acc: 0.0000e+00
Epoch 4717/5000
488/488 [==============================] - 0s - loss: 0.0311 - acc: 0.4597 - val_loss: 1.8498 - val_acc: 0.0000e+00
Epoch 4718/5000
488/488 [==============================] - 0s - loss: 0.0313 - acc: 0.4638 - val_loss: 1.8961 - val_acc: 0.0000e+00
Epoch 4719/5000
488/488 [==============================] - 0s - loss: 0.0314 - acc: 0.4590 - val_loss: 1.8382 - val_acc: 0.0000e+00
Epoch 4720/5000
488/488 [==============================] - 0s - loss: 0.0317 - acc: 0.4590 -

488/488 [==============================] - 0s - loss: 0.0306 - acc: 0.4631 - val_loss: 1.8537 - val_acc: 0.0000e+00
Epoch 4776/5000
488/488 [==============================] - 0s - loss: 0.0307 - acc: 0.4604 - val_loss: 1.8821 - val_acc: 0.0000e+00
Epoch 4777/5000
488/488 [==============================] - 0s - loss: 0.0309 - acc: 0.4611 - val_loss: 1.8255 - val_acc: 0.0000e+00
Epoch 4778/5000
488/488 [==============================] - 0s - loss: 0.0310 - acc: 0.4590 - val_loss: 1.9052 - val_acc: 0.0000e+00
Epoch 4779/5000
488/488 [==============================] - 0s - loss: 0.0312 - acc: 0.4617 - val_loss: 1.8179 - val_acc: 0.0000e+00
Epoch 4780/5000
488/488 [==============================] - 0s - loss: 0.0312 - acc: 0.4597 - val_loss: 1.8939 - val_acc: 0.0000e+00
Epoch 4781/5000
488/488 [==============================] - 0s - loss: 0.0312 - acc: 0.4631 - val_loss: 1.8584 - val_acc: 0.0000e+00
Epoch 4782/5000
488/488 [==============================] - 0s - loss: 0.0308 - acc: 0.4624 -

488/488 [==============================] - 0s - loss: 0.0308 - acc: 0.4617 - val_loss: 1.8800 - val_acc: 0.0000e+00
Epoch 4838/5000
488/488 [==============================] - 0s - loss: 0.0306 - acc: 0.4617 - val_loss: 1.8684 - val_acc: 0.0000e+00
Epoch 4839/5000
488/488 [==============================] - 0s - loss: 0.0304 - acc: 0.4570 - val_loss: 1.8767 - val_acc: 0.0000e+00
Epoch 4840/5000
488/488 [==============================] - 0s - loss: 0.0305 - acc: 0.4617 - val_loss: 1.8661 - val_acc: 0.0000e+00
Epoch 4841/5000
488/488 [==============================] - 0s - loss: 0.0306 - acc: 0.4597 - val_loss: 1.8452 - val_acc: 0.0000e+00
Epoch 4842/5000
488/488 [==============================] - 0s - loss: 0.0304 - acc: 0.4604 - val_loss: 1.8733 - val_acc: 0.0000e+00
Epoch 4843/5000
488/488 [==============================] - 0s - loss: 0.0303 - acc: 0.4645 - val_loss: 1.8700 - val_acc: 0.0000e+00
Epoch 4844/5000
488/488 [==============================] - 0s - loss: 0.0302 - acc: 0.4617 -

488/488 [==============================] - 0s - loss: 0.0306 - acc: 0.4570 - val_loss: 1.8436 - val_acc: 0.0000e+00
Epoch 4900/5000
488/488 [==============================] - 0s - loss: 0.0306 - acc: 0.4658 - val_loss: 1.8766 - val_acc: 0.0000e+00
Epoch 4901/5000
488/488 [==============================] - 0s - loss: 0.0306 - acc: 0.4577 - val_loss: 1.8409 - val_acc: 0.0000e+00
Epoch 4902/5000
488/488 [==============================] - 0s - loss: 0.0306 - acc: 0.4631 - val_loss: 1.8806 - val_acc: 0.0000e+00
Epoch 4903/5000
488/488 [==============================] - 0s - loss: 0.0307 - acc: 0.4597 - val_loss: 1.8486 - val_acc: 0.0000e+00
Epoch 4904/5000
488/488 [==============================] - 0s - loss: 0.0308 - acc: 0.4617 - val_loss: 1.8811 - val_acc: 0.0000e+00
Epoch 4905/5000
488/488 [==============================] - 0s - loss: 0.0309 - acc: 0.4624 - val_loss: 1.8481 - val_acc: 0.0000e+00
Epoch 4906/5000
488/488 [==============================] - 0s - loss: 0.0311 - acc: 0.4583 -

488/488 [==============================] - 0s - loss: 0.0305 - acc: 0.4624 - val_loss: 1.8674 - val_acc: 0.0000e+00
Epoch 4962/5000
488/488 [==============================] - 0s - loss: 0.0307 - acc: 0.4652 - val_loss: 1.8626 - val_acc: 0.0000e+00
Epoch 4963/5000
488/488 [==============================] - 0s - loss: 0.0310 - acc: 0.4631 - val_loss: 1.8612 - val_acc: 0.0000e+00
Epoch 4964/5000
488/488 [==============================] - 0s - loss: 0.0310 - acc: 0.4604 - val_loss: 1.8653 - val_acc: 0.0000e+00
Epoch 4965/5000
488/488 [==============================] - 0s - loss: 0.0309 - acc: 0.4652 - val_loss: 1.8604 - val_acc: 0.0000e+00
Epoch 4966/5000
488/488 [==============================] - 0s - loss: 0.0306 - acc: 0.4590 - val_loss: 1.8743 - val_acc: 0.0000e+00
Epoch 4967/5000
488/488 [==============================] - 0s - loss: 0.0304 - acc: 0.4617 - val_loss: 1.8612 - val_acc: 0.0000e+00
Epoch 4968/5000
488/488 [==============================] - 0s - loss: 0.0303 - acc: 0.4638 -

In [129]:
model.save('{}_MODEL.h5'.format(TICKER), overwrite=True)  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

In [130]:
FITTED = model.predict(TRAIN_X)
TEST_PREDICTION = model.predict(TEST_X)

In [131]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import Jitter
output_notebook()

Loading BokehJS ...

In [132]:
x_range = list(range(TRAIN_X.shape[0]))

p1 = figure(title="Fitted vs. Actual Returns" , plot_width=900 , plot_height=300)
p1.line( x=x_range , y=TRAIN_Y[:,0,0].flatten().tolist() , alpha=0.5 , legend="Target")
p1.line( x=x_range , y=FITTED[:,0,0].flatten().tolist() , color='red' , alpha=0.5 , legend="Fitted")

p2 = figure(title="Fitted Same Direction as Actual" , plot_width=900 , plot_height=300)
p2.circle( x=x_range , alpha=0.5 , y=((FITTED>0.0) == (TRAIN_Y>0.0))[:,0,1].flatten().tolist() )

p3 = figure(title="Fitted Returns - Actual Returns" , plot_width=900 , plot_height=300)
p3.circle( x=x_range , y=( FITTED - TRAIN_Y )[:,0,0].flatten().tolist() , color='orange' , alpha=0.5)

p4 = figure(title="Test Predicted vs. Target" , plot_width=900 , plot_height=300)
p4.circle( x=TEST_PREDICTION[0,0,:] , y=TEST_Y[0,0,:] , color='red' , alpha=0.5)

p1.x_range = p2.x_range = p3.x_range

show(column(p1,p2,p3,p4))

In [133]:
# accuracy by horizon time step
for i in range( horizon ):
    print( ((FITTED>0) == (TRAIN_Y>0))[:,0,i].astype(int).flatten().sum()/TRAIN_Y.shape[0] )

0.512295081967
0.477459016393
0.483606557377
0.524590163934
0.518442622951
0.475409836066
0.491803278689
0.497950819672
0.510245901639
0.524590163934
0.536885245902
0.510245901639
0.481557377049
0.504098360656
0.524590163934
0.489754098361
0.479508196721
0.459016393443
0.481557377049
0.479508196721
0.483606557377
0.487704918033
0.491803278689
0.487704918033
0.47131147541
0.463114754098
0.508196721311
0.463114754098
0.489754098361
0.456967213115


In [134]:
for i in range( horizon ):
    print( ((TEST_PREDICTION>0) == (TEST_Y>0))[:,0,i].astype(int).flatten().sum()/TEST_Y.shape[0] )

0.0
0.0
1.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
1.0


In [135]:
(TEST_Y - TEST_PREDICTION)/TEST_Y

array([[[  1.17991275e+00,   1.87043260e+00,   2.78716602e-01,
           8.78743604e-01,  -3.22002164e+00,   1.04111501e-01,
           3.48165489e+00,   1.73931322e+00,  -1.19711554e-01,
          -2.83650995e+00,  -3.11559629e+00,  -1.49714900e+00,
           3.72414801e+00,  -1.53442128e+00,   2.26620515e+00,
          -1.43060189e+00,   3.85079783e-01,   4.54012678e-01,
           2.18778058e+00,   1.24417363e+00,  -1.61758311e+00,
          -3.39147981e+00,  -2.87470154e-01,  -2.02706333e+00,
          -6.50401755e+00,   1.62631408e-02,   4.78461071e+00,
           4.24935907e+00,   5.27501665e+01,   9.12670348e-01],
        [  1.03366804e+00,   4.18453758e+00,   9.50489668e-01,
           5.13474560e-01,   5.69728554e-01,   7.66606778e-01,
           7.11309783e-01,   5.33497713e-01,   7.98220992e-01,
           1.01577192e+00,   1.02297744e+00,   1.01371317e+00,
           6.81461542e-01,   7.93998694e-01,   1.49629298e+00,
           1.27856529e+00,   9.98313278e-01,   9.53459